In [1]:
import mediapipe as mp

#imports open cv
import cv2
import numpy as np

#special library which gives the coordinates of ur pinky finger ,palm
from mediapipe.framework.formats import landmark_pb2

import time
import random

In [2]:
#creating class instances
mp_drawing = mp.solutions.drawing_utils
mp_hands=mp.solutions.hands #basically all hand values
score=0
best_score = 0  # To keep track of the best score
palm_detected = False

In [3]:
x_enemy=random.randint(50,600)
y_enemy=random.randint(50,400)
game_started = False  # Timer starts only after pressing 'Start'

#circles that users has to touch to score
def enemy():
    global score,x_enemy,y_enemy
    # Ensure the enemy circle appears below the score area
    min_y_position = 200  # Set a minimum y-coordinate to avoid overlapping with the score text
    if x_enemy == -1 and y_enemy == -1:  # Initially set enemy position to -1 to avoid random placement
        x_enemy = random.randint(50, imageWidth - 50)
        y_enemy = random.randint(min_y_position, imageHeight - 50)

    # Draw the enemy circle
    cv2.circle(image, (x_enemy, y_enemy), 25, (0, 200, 0), 5)

In [4]:
#input from webcam
video=cv2.VideoCapture(0)

# Increase webcam feed size
video.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)  # Set width to 1280 pixels
video.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)  # Set height to 720 pixels

True

In [5]:
#min_detection_confidence- how sensitive the program will be to detect the features of hand
#more the value more the wrong detection

#min_tracking_confidence-sensitive of tracking or movements of the points

with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5) as hands:
    while video.isOpened():
        ret, frame = video.read()
        if not ret:
            break

        # Flip the frame for a mirrored view
        image = cv2.flip(frame, 1)
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        imageHeight, imageWidth, _ = image.shape

        # If the game hasn't started, display a black screen and prompt
        if not game_started:
            # Black screen
            image[:] = 0

            # Prompt message
            font = cv2.FONT_HERSHEY_SIMPLEX
            cv2.putText(image, "Show your palm to start", (300, 300), font, 1.5, (0, 255, 0), 4, cv2.LINE_AA)
            results = hands.process(image_rgb)
            
            # Check for palm detection
            if results.multi_hand_landmarks:
                palm_detected = True

            if palm_detected:
                cv2.putText(image, "Palm detected! Starting game...", (300, 350), font, 1, (0, 255, 255), 3, cv2.LINE_AA)
                cv2.imshow('Hand Tracking', image)
                cv2.waitKey(2000)  # Brief pause before game starts
                start_time = time.time()
                game_started = True
                score=0
                continue

            cv2.imshow('Hand Tracking', image)
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
            continue

        # Once game starts
        results = hands.process(image_rgb)
        image[:] = 0  # Keep the background black
        
        # Display scores and timer
        elapsed_time = int(time.time() - start_time)
        remaining_time = 60 - elapsed_time
        if remaining_time <= 0:
            print("Time's Up! Final Score:", score)
            if score > best_score:
                best_score = score
            game_started = False
            palm_detected = False
            continue

        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(image, f"Score: {score}", (50, 50), font, 1.5, (51,153,255), 3, cv2.LINE_AA)
        cv2.putText(image, f"Time: {remaining_time}s", (50, 100), font, 1.5, (255,0,255), 3, cv2.LINE_AA)
        cv2.putText(image, f"Best Score: {best_score}", (900, 50), font, 1.5, (0, 255, 255), 3, cv2.LINE_AA)

        # Draw enemy circle
        enemy()

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                # Draw the hand landmarks and connections (skeleton)
                mp_drawing.draw_landmarks(
                    image, hand_landmarks, mp_hands.HAND_CONNECTIONS,
                    mp_drawing.DrawingSpec(color=(0, 255, 255), thickness=2, circle_radius=2)
                )

                # Detect index finger tip
                index_finger_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
                # Convert the normalized coordinates (ranging from 0 to 1) to pixel coordinates
                pixel_coordinates = mp_drawing._normalized_to_pixel_coordinates(
                    index_finger_tip.x, index_finger_tip.y, imageWidth, imageHeight
                )

                # Check if the converted pixel coordinates are not None (i.e., valid coordinates)
                if pixel_coordinates is not None:
                    cv2.circle(image, (pixel_coordinates[0], pixel_coordinates[1]), 15, (0, 0, 255), 3)
                    
                    # Calculate the distance between the index finger tip and the target circle
                    distance = np.sqrt((pixel_coordinates[0] - x_enemy) ** 2 + (pixel_coordinates[1] - y_enemy) ** 2)

                    # Check if the index finger is within a certain distance from the target
                    if distance <= 30:
                        print("Target Hit!")
                        # Reposition the target after the user hits it
                        x_enemy = random.randint(50, imageWidth - 50)
                        y_enemy = random.randint(200, imageHeight - 50)  # Ensures target stays below the score
                        score += 1

        cv2.imshow('Hand Tracking', image)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

video.release()
cv2.destroyAllWindows()

Target Hit!
Target Hit!
Target Hit!
Target Hit!
Target Hit!
Target Hit!
Target Hit!
Target Hit!
Target Hit!
Target Hit!
Target Hit!
Target Hit!
Target Hit!
Target Hit!
Target Hit!
Target Hit!
Target Hit!
Target Hit!
Target Hit!
Target Hit!
Target Hit!
Target Hit!
Target Hit!
Target Hit!
Target Hit!
Target Hit!
Target Hit!
Target Hit!
Target Hit!
Time's Up! Final Score: 29
Target Hit!
Target Hit!
Target Hit!
Target Hit!
Time's Up! Final Score: 4
Target Hit!
Target Hit!
Target Hit!
Target Hit!
Target Hit!
Target Hit!
Target Hit!
Target Hit!
Target Hit!
Target Hit!
Target Hit!
Target Hit!
Target Hit!
Time's Up! Final Score: 13
Target Hit!
Target Hit!
Target Hit!
Target Hit!
